In [1]:
%load_ext autoreload
%autoreload 1
%aimport my

import sys
import numpy as np
import pandas as pd
import os
import gc

import my
from my import p
import text_utils

pd.set_option("max_colwidth", 25)
pd.set_option("display.precision", 1)
pd.options.display.float_format = "{:.3f}".format

from IPython.display import HTML, display

import requests
from bs4 import BeautifulSoup
import text_utils
     
dir_out = "out/"
dir_data = 'data/'
os.makedirs(dir_out, exist_ok = True)
RANDOM_STATE = 34

np.random.seed(RANDOM_STATE)
N_CPU = os.cpu_count()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
train = pd.read_csv(dir_data+'train_dataset_train.csv',parse_dates=['publish_date'])
test = pd.read_csv(dir_data+'test_dataset_test.csv',parse_dates=['publish_date'])
data = pd.concat([train,test]).reset_index(drop=True)
p(data.shape)

(10000, 11) 
 ~


### Парсинг страниц новостей
Путем визуального осмотра установлено, что первые 24 символа document_id определяют часть URL страницы новости. Определим URL для всех новостей.

In [3]:
data['path'] = data['document_id'].str[:24]
data[:2]

,document_id,title,publish_date,session,authors,ctr,category,tags,views,depth,full_reads_percent,path
0,624ac09c9a7947db3d80c...,Европейский банк разв...,2022-04-04 10:29:44,IDE7mtH4RBqGn-8MXfGffQ,[],1.580,5409f11ce063da9c8b588a18,['55928d339a794751dc8...,20460.000,1.134,35.850,624ac09c9a7947db3d80c98e
1,620f6b899a7947701cf48...,Кремль назвал регуляр...,2022-02-18 10:00:39,KtVJsteHStO5oditt3Uvzw,['54244e01cbb20f03076...,1.853,5409f11ce063da9c8b588a12,['549d25df9a794775979...,19038.000,1.142,38.355,620f6b899a7947701cf489e1


In [4]:
data['url'] = 'https://www.rbc.ru/rbcfreenews/' + data['path']
list_urls = data[['path','url']].drop_duplicates().to_numpy().tolist()
len(list_urls),list_urls[:3]

(9890,
 [['624ac09c9a7947db3d80c98e',
   'https://www.rbc.ru/rbcfreenews/624ac09c9a7947db3d80c98e'],
  ['620f6b899a7947701cf489e1',
   'https://www.rbc.ru/rbcfreenews/620f6b899a7947701cf489e1'],
  ['620730cf9a7947ab96a44e27',
   'https://www.rbc.ru/rbcfreenews/620730cf9a7947ab96a44e27']])

In [5]:
my.save_pickle(dir_data+'urls.pickle',list_urls)

In [6]:
# Функция для извлечения признаков из html страницы новости
def stat_article(info_page):
    i, _, pg = info_page

    bs = BeautifulSoup(pg, 'html.parser')

    # находим элемент статьи
    article = bs.find('div',class_='article__text')

    st = {'path':i,
    'ww_number_links':0,
    'ww_have_overview':0,
    'ww_have_ticker': 0,
    'ww_len_overview':0,
    'ww_num_symbols': 0,
    'ww_num_symbols_until_item': 0,
    'ww_pro_in_article':0,
    'ww_num_inline_items':0,
    'ww_have_iframe': 0,

    'ww_number_ps':0,
    'ww_num_symbols_until_pro':0,
    'ww_have_video_recommend':0,
    'ww_have_video_in': 0,
    'ww_num_symbols_until_video_in':0,
    'ww_have_image': 0,
    'text': []
    }

    # перебираем все составляющие элементы статьи и вычисляем их количество, характеристики

    st['ww_number_links'] = len(article.find_all('a',class_=''))
    st['ww_number_ps'] = len(article.find_all('p',class_=''))

    # есть ли обзор
    elem = article.find('div',class_='article__text__overview')
    if elem:
        st['ww_have_overview'] = 1
        st['ww_len_overview'] = len(elem.text)
        st['ww_num_symbols'] += len(elem.text)
        st['text'].append(elem.text)

    # есть ли курсы валют в статье
    elem = article.find('div',class_='article__ticker')
    if elem:
        st['ww_have_ticker'] = 1

    # есть ли изображение
    elem = article.find('div',class_='article__main-image')
    if elem:
        st['ww_have_image'] = 1

    # есть ли встроенные объект
    elem = article.find('iframe')
    if elem:
        st['ww_have_iframe'] = 1

    elem = article.find('div',class_='video-autoplay-recommend')
    if elem:
        st['ww_have_video_recommend'] = 1

    # количество элементов, ведущих на другие новости
    elem = article.find_all('div',class_='article__inline-item')
    if elem:
        st['ww_num_inline_items'] = len(elem)

    pro_anons = False
    video_in = False

    for ch in article.children:
        if ch.name == None:
            continue
        
        # подсчитываем количество символов на разных этапах статьи
        if ch.name in ['p','ul']:
            if pro_anons and st['ww_pro_in_article'] == 0:
                st['ww_pro_in_article'] = 1
                st['ww_num_symbols_until_pro'] = st['ww_num_symbols']

            if video_in and st['ww_have_video_in'] == 0:
                st['ww_have_video_in'] = 1
                st['ww_num_symbols_until_video_in'] = st['ww_num_symbols']

            if ch.find('div',class_='article__inline-item') and st['ww_num_symbols_until_item']==0:
                st['ww_num_symbols_until_item'] = st['ww_num_symbols']

            elif len(ch.text) > 20:
                st['ww_num_symbols'] += len(ch.text)
                st['text'].append(ch.text)

        if ch.name == 'div' and ch.has_attr('class') and ch['class'][0] == 'article__special_container':
            st['ww_num_symbols'] += len(ch.text)
            

        if ch.name == 'div' and ch.has_attr('class') and ch['class'][0] == 'pro-anons':
            pro_anons = True

        if ch.name == 'div' and ch.has_attr('class') and ch['class'][0] == 'fox-tail':
            video_in = True
            

    if st['ww_num_symbols_until_item'] == 0:
        st['ww_num_symbols_until_item'] = st['ww_num_symbols']

    if st['ww_num_symbols_until_pro'] == 0:
        st['ww_num_symbols_until_pro'] = st['ww_num_symbols']
   
    if st['ww_num_symbols_until_video_in'] == 0:
        st['ww_num_symbols_until_video_in'] = 10000

    st['ww_feat1'] = st['ww_num_symbols_until_item']/(st['ww_num_symbols']+1)
    st['ww_feat2'] = st['ww_num_symbols_until_pro']/(st['ww_num_symbols']+1)
    st['ww_feat3'] = st['ww_num_symbols_until_video_in']/(st['ww_num_symbols']+1)

    st['text'] = ' '.join(st['text'])[:1000].replace('\n','').replace('\r','')
   
    return st

### парсим страницы

Выполняем запрос к странице сайта, получаем html страницы, извлекаем признаки из нее функцией stat_article, сохраняем результат в файл pages_info.p.

In [ ]:
res = []
headers = requests.utils.default_headers()

headers.update(
        {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 YaBrowser/20.9.3.136 Yowser/2.5 Safari/537.36',
        })

s = requests.Session()

for i in range(0,len(list_urls)):
    
    if i % 50 == 0:
        s = requests.Session()
            
    response = s.get(list_urls[i][1],headers=headers)

    if response.status_code != 200:
        print('ОШИБКА', response.status_code, response.text)
        break
    
    res.append(stat_article((list_urls[i][0],response.url,response.text)))

In [ ]:
res = pd.DataFrame(res)
res['text'] = text_utils.clean_text(res['text'],do_lemma=True)
res.drop_duplicates(subset=['path'],inplace=True)
res.set_index('path',inplace=True)
res.to_parquet(dir_data+'pages_info.p')
p(res.shape)
res[:5]